In [18]:
import pandas as pd
import difflib,nltk
from IPython.core.display import display, HTML

dataset = pd.read_csv('data/preprints_full_20190901_20200430.csv')
print (len(dataset))

25883


In [19]:
dataset.head()

,source,doi,posted_date,covid_preprint,title,abstract,n_versions,license,type,category,...,published_journal_is_oa,published_best_oa_location,published_best_oa_license,n_words,n_refs,institution_match_score,institution_match_type,institution_match_name,institution_match_country_name,institution_match_country_code
0,medrxiv,10.1101/19000828,2019-10-25,False,Association Between Blood Circulating Vitamin ...,Objectives: We conducted this dose-response me...,1,cc_by_nc_nd,PUBLISHAHEADOFPRINT,oncology,...,True,publisher,cc-by-nc,NaN,NaN,1.0,PHRASE,Peking Union Medical College Hospital,China,CN
1,medrxiv,10.1101/19002394,2019-09-16,False,CX3CR1 as a Respiratory Syncytial Virus Recept...,Objectives: Respiratory Syncytial Virus (RSV) ...,1,cc_by_nc_nd,PUBLISHAHEADOFPRINT,pediatrics,...,False,repository,NaN,NaN,NaN,1.0,PHRASE,University of Rochester,United States,US
2,medrxiv,10.1101/19003327,2019-09-12,False,Alcohol use and cognitive functioning in young...,Background and Aims: There have been few longi...,3,cc_by,PUBLISHAHEADOFPRINT,epidemiology,...,False,publisher,cc-by,NaN,NaN,1.0,PHRASE,University of Bristol,United Kingdom,GB
3,medrxiv,10.1101/19003855,2019-09-04,False,Identifying 31 novel breast cancer susceptibil...,Common genetic variants in 183 loci have been ...,1,cc_by_nc_nd,PUBLISHAHEADOFPRINT,epidemiology,...,NaN,NaN,NaN,NaN,NaN,1.0,PHRASE,Vanderbilt University Medical Center,United States,US
4,medrxiv,10.1101/19003889,2019-09-20,False,Does in utero HIV-exposure influence infant de...,Background: As mother-to-child-transmission of...,3,cc_no,PUBLISHAHEADOFPRINT,hiv aids,...,NaN,NaN,NaN,NaN,NaN,1.0,PHRASE,Carleton University,Canada,CA


In [20]:
def show_diff(text, n_text):
    """
    Based upon http://stackoverflow.com/a/788780
    Unify operations between two compared strings seqm is a difflib.
    SequenceMatcher instance whose a & b are strings
    """
    
    seqm = difflib.SequenceMatcher(None, text, n_text)
    ratio = float(1.0 - seqm.ratio())
    output= []
    removals = get_all_removals(abs1,abs2)
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'equal':
            output.append(seqm.a[a0:a1])
        elif opcode == 'insert':
            check_rearr = find_rearrangements(removals,seqm.b[b0:b1],"green")
            output.append(check_rearr )
        elif opcode == 'delete':
            output.append("<strike><font color=red>" + seqm.a[a0:a1] + "</font></strike>")
        elif opcode == 'replace':
            # seqm.a[a0:a1] -> seqm.b[b0:b1]
            check_rearr = find_rearrangements(removals,seqm.b[b0:b1],"blue")
            output.append("<strike><font color=red>"+ seqm.a[a0:a1] + "</font></strike> "+ check_rearr )
        else:
            raise (RuntimeError, "unexpected opcode")
    return ratio, ''.join(output)

In [21]:
def get_all_removals(text, n_text):
    seqm = difflib.SequenceMatcher(None, text, n_text)
    output= []    
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'delete':
            output.append(seqm.a[a0:a1])
        elif opcode == 'replace':
            output.append(seqm.a[a0:a1])
    return ''.join(output)  

ngram_up_to = 2
def get_all_ngrams(text,ngram_up_to):
    """Returns all ngrams from a text up to a certain number.
    
    Args:
        text: a string.
        ngram_up_to: a integer.
    Returns:
        A list of ngrams.
    """
    
    tokens = text.split(" ")
    ngrams = [" ".join(x) for n in range(2,ngram_up_to+1) for x in nltk.ngrams(tokens,n)]  
    match_ngrams_tokens = {}
    for x in range(len(tokens)):
        token = tokens[x]
        match_ngrams_tokens[x] = [x-1,x]
    return ngrams,match_ngrams_tokens,tokens

def find_rearrangements(removals,snippet,colour):
    
    ngrams,match_ngrams_tokens,tokens = get_all_ngrams(snippet,ngram_up_to)

    output= []    
    
    for t in range(len(tokens)):
        token = tokens[t]
        rel_ngrams = match_ngrams_tokens[t]
        check = False
        for ngram_id in rel_ngrams:
            try:
                rel_ngram = ngrams[ngram_id]
                if rel_ngram in removals:
                    check = True
            except Exception as e:
                continue
        if check == True:
            output.append("<font color=orange>" + token+ " </font>")
        else:    
            output.append("<font color="+colour+">" + token+ " </font>")
        
    return ''.join(output)    

In [22]:
def display_changes(doi):
    for index, row in dataset.iterrows():
        if doi == row["doi"]:
            abs1 = row["abstract"]
            abs2 = row["published_abstract"]
            if type(abs2) is not str:
                try:
                    abs2 = row["published_pubmed_abstract"]
                except Exception:
                    return "Missing Final Abstract"
        #        number = row["Number"]

            change_ratio, out = show_diff(abs1,abs2)
            print ("Computed change ratio:", change_ratio)
            print ()
            display(HTML(out))
                    
display_changes("10.1101/19000828")

Computed change ratio: 0.48739320691810795



In [13]:
for index, row in dataset.iterrows():
    #abs1 = row["v1 Preprint abstract"]
    #abs2 = row["Paper abstract"]
    abs1 = row["abstract"]
    abs2 = row["published_abstract"]
    if type(abs2) is not str:
        try:
            abs2 = row["published_pubmed_abstract"]
        except Exception:
            continue
#        number = row["Number"]
    
    try:
            change_ratio, out = show_diff(abs1,abs2)
            if  change_ratio> 0.0:
                #print ("Number: ",number)
                #print ("Scored abstract change:", row["Scored abstract change"])
                print ("Computed change ratio:", change_ratio)
                print ()
    #            print (" ")
    #            print (abs1)
    #            print (" ")
    #            print (abs2)
                print (" ")
                display(HTML(out))
                print (" ")
                print (row["doi"])

    except Exception as e:
            print (e,type(row["published_abstract"]),type(row["published_pubmed_abstract"]))
            continue

Computed change ratio: 0.48739320691810795

 


 
10.1101/19000828
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
Computed change ratio: 0.2348090277777778

 


 
10.1101/19004218
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
Computed change ratio: 0.35109458901280466

 


 
10.1101/19005421
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
Computed change ratio: 0.698773166275124

 


 
10.1101/19005652
'float' object is not iterable <class 'float'> <class 'float'>
Computed change ratio: 0.04930372600677457

 


...

 
10.1101/2019.12.16.877464
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
Computed change ratio: 0.16459197786998614

 


 
10.1101/2019.12.16.877506
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
Computed change ratio: 0.008216529724504595

 


 
10.1101/2019.12.16.877605
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
Computed change ratio: 0.576387445437539

 


 
10.1101/2019.12.16.877779
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
'float' object is not iterable <class 'float'> <class 'float'>
Computed change ratio: 0.06307977736549164

 


KeyboardInterrupt: 